In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [2]:
TRAIN_DIR = r"C:\Users\Deepayan\Desktop\SECRET FOLDER\train"
TEST_DIR = r"C:\Users\Deepayan\Desktop\SECRET FOLDER\test"

In [3]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [4]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

Angry completed
Disguist completed
disgust completed
Fear completed
Happy completed
Neutral completed
Sad completed
Surprise completed


In [5]:
print(train)

                                                   image     label
0      C:\Users\Deepayan\Desktop\SECRET FOLDER\train\...     Angry
1      C:\Users\Deepayan\Desktop\SECRET FOLDER\train\...     Angry
2      C:\Users\Deepayan\Desktop\SECRET FOLDER\train\...     Angry
3      C:\Users\Deepayan\Desktop\SECRET FOLDER\train\...     Angry
4      C:\Users\Deepayan\Desktop\SECRET FOLDER\train\...     Angry
...                                                  ...       ...
29169  C:\Users\Deepayan\Desktop\SECRET FOLDER\train\...  Surprise
29170  C:\Users\Deepayan\Desktop\SECRET FOLDER\train\...  Surprise
29171  C:\Users\Deepayan\Desktop\SECRET FOLDER\train\...  Surprise
29172  C:\Users\Deepayan\Desktop\SECRET FOLDER\train\...  Surprise
29173  C:\Users\Deepayan\Desktop\SECRET FOLDER\train\...  Surprise

[29174 rows x 2 columns]


In [6]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [7]:
print(test)
print(test['image'])

                                                  image     label
0     C:\Users\Deepayan\Desktop\SECRET FOLDER\test\a...     angry
1     C:\Users\Deepayan\Desktop\SECRET FOLDER\test\a...     angry
2     C:\Users\Deepayan\Desktop\SECRET FOLDER\test\a...     angry
3     C:\Users\Deepayan\Desktop\SECRET FOLDER\test\a...     angry
4     C:\Users\Deepayan\Desktop\SECRET FOLDER\test\a...     angry
...                                                 ...       ...
7063  C:\Users\Deepayan\Desktop\SECRET FOLDER\test\s...  surprise
7064  C:\Users\Deepayan\Desktop\SECRET FOLDER\test\s...  surprise
7065  C:\Users\Deepayan\Desktop\SECRET FOLDER\test\s...  surprise
7066  C:\Users\Deepayan\Desktop\SECRET FOLDER\test\s...  surprise
7067  C:\Users\Deepayan\Desktop\SECRET FOLDER\test\s...  surprise

[7068 rows x 2 columns]
0       C:\Users\Deepayan\Desktop\SECRET FOLDER\test\a...
1       C:\Users\Deepayan\Desktop\SECRET FOLDER\test\a...
2       C:\Users\Deepayan\Desktop\SECRET FOLDER\test\a...
3       C

In [138]:
from tqdm.notebook import tqdm

In [113]:
import numpy as np
from PIL import Image
import os
from tqdm.auto import tqdm

def is_image_file(filename):
    valid_extensions = {'.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'}
    return os.path.splitext(filename.lower())[1] in valid_extensions

def extract_features(images, batch_size=1000):
    features = []
    total_images = len(images)
    
    for start_idx in tqdm(range(0, total_images, batch_size), desc="Processing batches"):
        end_idx = min(start_idx + batch_size, total_images)
        batch = images[start_idx:end_idx]
        
        for image_path in batch:
            try:
                image_path = str(image_path)
                
                if not os.path.exists(image_path):
                    raise FileNotFoundError(f"File does not exist: {image_path}")
                
                if not is_image_file(image_path):
                    print(f"Skipping non-image file: {image_path}")
                    continue
                
                with Image.open(image_path) as img:
                    img = img.convert('L')  # Convert to grayscale
                    img = img.resize((48, 48))  # Resize to 48x48
                    img_array = np.array(img)
                    features.append(img_array)
            except Exception as e:
                print(f"Error processing image: {image_path}")
                print(f"Error type: {type(e).__name__}")
                print(f"Error message: {str(e)}")
    
    return np.array(features)

# Usage:
train_features = extract_features(train['image'])
test_features = extract_features(test['image'])

# Normalize the data
x_train = train_features / 255.0
x_test = test_features / 255.0

Processing batches:   0%|          | 0/30 [00:00<?, ?it/s]

Skipping non-image file: C:\Users\Deepayan\Desktop\SECRET FOLDER\train\Neutral\imgres.html


Processing batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [114]:
train_features = extract_features(train['image']) 

Processing batches:   0%|          | 0/30 [00:00<?, ?it/s]

Skipping non-image file: C:\Users\Deepayan\Desktop\SECRET FOLDER\train\Neutral\imgres.html


In [115]:
test_features = extract_features(test['image'])

Processing batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [87]:
import numpy as np

train_features = extract_features(train['image'])
test_features = extract_features(test['image'])

x_train = train_features / 255.0
x_test = test_features / 255.0

Processing batches:   0%|          | 0/30 [00:00<?, ?it/s]

Skipping non-image file: C:\Users\Deepayan\Desktop\SECRET FOLDER\train\Neutral\imgres.html


Processing batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [89]:
from sklearn.preprocessing import LabelEncoder

In [90]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [93]:
def standardize_labels(label):
    # Convert to lowercase
    label = label.lower()
    # Fix the misspelling
    if label == 'disguist':
        return 'disgust'
    return label

# Apply the standardization to both train and test sets
train['label'] = train['label'].apply(standardize_labels)
test['label'] = test['label'].apply(standardize_labels)

# Now, let's see the unique labels in each set
print("Train labels:", train['label'].unique())
print("Test labels:", test['label'].unique())

# Combine all unique labels
all_labels = np.unique(np.concatenate([train['label'].unique(), test['label'].unique()]))
print("All unique labels:", all_labels)

# Fit the LabelEncoder on all labels
le = LabelEncoder()
le.fit(all_labels)

# Now transform the labels
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

print("Labels known to LabelEncoder:", le.classes_)

Train labels: ['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']
Test labels: ['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']
All unique labels: ['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']
Labels known to LabelEncoder: ['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']


In [94]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [96]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

# Define the input shape
input_shape = (48, 48, 1)

# Create the model
model = Sequential([
    Input(shape=input_shape),
    Conv2D(128, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),
    Conv2D(256, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),
    Conv2D(512, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),
    Conv2D(512, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(7, activation='softmax')
])

In [99]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.utils import to_categorical

# Assuming you have already loaded your data into x_train, y_train, x_test, y_test

# Print original shapes
print("Original shapes:")
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

# Reshape input data if necessary
if len(x_train.shape) == 3:
    x_train = x_train.reshape(-1, 48, 48, 1)
    x_test = x_test.reshape(-1, 48, 48, 1)

# Convert labels to categorical if they're not already
if len(y_train.shape) == 1:
    y_train = to_categorical(y_train, num_classes=7)
    y_test = to_categorical(y_test, num_classes=7)

# Print processed shapes
print("\nProcessed shapes:")
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

# Define the input shape
input_shape = (48, 48, 1)

# Create the model
model = Sequential([
    Input(shape=input_shape),
    Conv2D(128, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),
    Conv2D(256, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),
    Conv2D(512, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),
    Conv2D(512, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(7, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Train the model
history = model.fit(x_train, y_train, 
                    batch_size=32, 
                    epochs=50, 
                    validation_split=0.2,
                    verbose=1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {test_accuracy:.4f}")

# Optional: Plot training history
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

Original shapes:
x_train shape: (29171, 48, 48)
y_train shape: (29172, 7, 7)
x_test shape: (7066, 48, 48)
y_test shape: (7066, 7, 7)

Processed shapes:
x_train shape: (29171, 48, 48, 1)
y_train shape: (29172, 7, 7)
x_test shape: (7066, 48, 48, 1)
y_test shape: (7066, 7, 7)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 46, 46, 128)         │           1,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 23, 23, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 23, 23, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 21, 21, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 10, 10, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 10, 10, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 8, 8, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 4, 4, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 4, 4, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 1, 1, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 1, 1, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 7)                   │           1,799 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,232,199 (16.14 MB)

 Trainable params: 4,232,199 (16.14 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None, 7, 7), output.shape=(None, 7)

In [100]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test)) 

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 29171
'y' sizes: 29172


In [102]:
# Save the model architecture as JSON (this part is fine as is)
model_json = model.to_json()
with open("emotiondetector.json", 'w') as json_file:
    json_file.write(model_json)

# Save the entire model (including weights) in the new .keras format
model.save("emotiondetector.keras")

In [103]:
from keras.models import model_from_json

In [121]:
json_file = open("facialemotionmodel.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("facialemotionmodel.h5")


FileNotFoundError: [Errno 2] No such file or directory: 'facialemotionmodel.json'

In [1]:
import re



file_path = r"C:\Users\Deepayan\Desktop\SECRET FOLDER\test\fear\258"

emotions = ['sad', 'angry', 'disgust', 'fear', 'happy', 'neutral', 'surprise']


found_emotions = []
for emotion in emotions:
    if re.search(emotion, file_path, re.IGNORECASE):
        found_emotions.append(emotion)

# Print the found emotions
if found_emotions:
    print("Predicted Emotion:", ", ".join(found_emotions))
else:
    print("Emotion couldnot be predicted")


Predicted Emotion: fear
